In [1]:
# I want to predict the individual teams total points and add them together so build dataframes with teams offensive stats and opponents defensive stats

# Retrieve data, clean data and store in folder to train model on

In [2]:
# Imports

import nflreadpy as nfl
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

## Retrieve data

In [3]:
schedule = nfl.load_schedules(seasons=2025)
schedule = schedule.to_pandas()

team_stats = nfl.load_team_stats(seasons=2025)
team_stats = team_stats.to_pandas()

In [4]:
team_stats

,season,week,team,season_type,opponent_team,completions,attempts,passing_yards,passing_tds,passing_interceptions,...,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance
0,2025,1,ARI,REG,NO,21,29,163,2,0,...,2,2,0,0,1.000000,0,0,0,0,0
1,2025,1,ATL,REG,TB,27,42,298,1,0,...,2,2,0,0,1.000000,0,0,0,0,0
2,2025,1,BAL,REG,BUF,14,19,209,2,0,...,4,5,1,0,0.800000,0,0,0,0,0
3,2025,1,BUF,REG,BAL,33,46,394,2,0,...,2,2,0,0,1.000000,1,1,0,0,32
4,2025,1,CAR,REG,JAX,18,35,154,1,2,...,1,1,0,0,1.000000,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,2025,10,PIT,REG,LAC,16,31,161,1,2,...,1,1,0,0,1.000000,0,0,0,0,0
292,2025,10,SEA,REG,ARI,10,12,178,1,1,...,5,5,0,0,1.000000,0,0,0,0,0
293,2025,10,SF,REG,LA,33,39,319,3,1,...,2,3,0,1,0.666667,0,0,0,0,0
294,2025,10,TB,REG,NE,28,43,273,3,0,...,2,2,0,0,1.000000,0,0,0,0,0


In [5]:
# Filter out data I don't want

cols_to_drop = ['season', 
 'season_type',
 'sack_fumbles',
 'passing_air_yards',
 'passing_yards_after_catch',
 'passing_first_downs',
 'passing_2pt_conversions',
 'rushing_2pt_conversions',
 'receptions',
 'targets',
 'receiving_yards',
 'receiving_tds',
 'receiving_fumbles',
 'receiving_air_yards',
 'receiving_yards_after_catch',
 'receiving_first_downs',
 'receiving_2pt_conversions',
 'special_teams_tds',
 'misc_yards',
 'fumble_recovery_own',
 'fumble_recovery_yards_own',
 'fumble_recovery_opp',
 'fumble_recovery_yards_opp',
 'fumble_recovery_tds',
 'timeouts',
 'fg_made',
 'fg_missed',
 'fg_blocked',
 'fg_long',
 'fg_made_0_19',
 'fg_made_20_29',
 'fg_made_30_39',
 'fg_made_40_49',
 'fg_made_50_59',
 'fg_made_60_',
 'fg_missed_0_19',
 'fg_missed_20_29',
 'fg_missed_30_39',
 'fg_missed_40_49',
 'fg_missed_50_59',
 'fg_missed_60_',
 'fg_made_list',
 'fg_missed_list',
 'fg_blocked_list',
 'fg_made_distance',
 'fg_missed_distance',
 'fg_blocked_distance', 
 'pat_made',
 'pat_att',
 'pat_missed',
 'pat_blocked',
 'gwfg_made',
 'gwfg_att',
 'gwfg_missed',
 'gwfg_blocked',
 'gwfg_distance']

team_stats = team_stats.drop(columns=cols_to_drop)

In [6]:
# for each week group by team sum/avg all previous weeks 
team_stats = team_stats.sort_values(["team", "week"]).reset_index(drop=True)

In [7]:
# Cumulative averages up to a certain week

avg_cols = ['passing_epa',
 'passing_cpoe',
 'rushing_epa',
 'receiving_epa',
 'fg_pct',
 'pat_pct',
 'completions',
 'attempts',
 'passing_yards',
 'passing_tds',
 'passing_interceptions',
 'sacks_suffered',
 'sack_yards_lost',
 'sack_fumbles_lost',
 'carries',
 'rushing_yards',
 'rushing_tds',
 'rushing_fumbles_lost',
 'rushing_first_downs',
 'receiving_fumbles_lost',
 'def_tackles_solo',
 'def_tackles_with_assist',
 'def_tackle_assists',
 'def_tackles_for_loss',
 'def_tackles_for_loss_yards',
 'def_fumbles_forced',
 'def_sacks',
 'def_sack_yards',
 'def_qb_hits',
 'def_interceptions',
 'def_interception_yards',
 'def_pass_defended',
 'def_tds',
 'def_fumbles',
 'def_safeties',
 'penalties',
 'penalty_yards',
 'punt_returns',
 'punt_return_yards',
 'kickoff_returns',
 'kickoff_return_yards',
 'fg_att']


In [8]:
team_stats['week'] = team_stats['week'].astype(int)

# Get all unique teams and full week range
teams = team_stats['team'].unique()
weeks = range(1, team_stats['week'].max() + 1)

# Create a MultiIndex of all possible (team, week) combinations
full_index = pd.MultiIndex.from_product([teams, weeks], names=['team', 'week'])

# Reindex the dataframe to include all rows
team_stats = team_stats.set_index(['team', 'week']).reindex(full_index).reset_index()

In [9]:
# Compute cumulative mean for previous games (excluding current game)
team_stats[[f"avg_{c}" for c in avg_cols]] = (
    team_stats.sort_values(by=['team', 'week'])
    .groupby("team")[avg_cols]
      .expanding()
      .mean()
      .shift(periods=1)
      .reset_index(level=0, drop=True)
)

## Retrieve Time of Possesion Stats

In [10]:
for attempt in range(1, 5 + 1):
    try:
        # set up the browser
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
        
        # visit T.O.P. Website
        driver.get("https://www.teamrankings.com/nfl/stat/average-time-of-possession-net-of-ot")

        table = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//*[@id='DataTables_Table_0']"))
                    )

        df = pd.read_html(table.get_attribute("outerHTML"))[0]

        driver.quit()
        break

    except Exception as e:
        driver.quit()
        print(f"Attempt {attempt} failed: {e}")
        if attempt == 5:
            print("No retries left — raising.")
            raise

C:\Users\tyler\AppData\Local\Temp\ipykernel_43192\26274015.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table.get_attribute("outerHTML"))[0]


In [11]:
mapping = {'Buffalo': 'BUF',
 'LA Chargers': 'LAC',
 'Kansas City': 'KC',
 'Houston': 'HOU',
 'Detroit': 'DET',
 'Carolina': 'CAR',
 'Arizona': 'ARI',
 'New England': 'NE',
 'Jacksonville': 'JAX',
 'Chicago': 'CHI',
 'Tampa Bay': 'TB',
 'Atlanta': 'ATL',
 'NY Giants': 'NYG',
 'Denver': 'DEN',
 'Green Bay': 'GB',
 'San Francisco': 'SF',
 'Philadelphia': 'PHI',
 'Cleveland': 'CLE',
 'Dallas': 'DAL',
 'NY Jets': 'NYJ',
 'Indianapolis': 'IND',
 'Las Vegas': 'LV',
 'Minnesota': 'MIN',
 'Seattle': 'SEA',
 'Miami': 'MIA',
 'LA Rams': 'LA',
 'Baltimore': 'BAL',
 'New Orleans': 'NO',
 'Tennessee': 'TEN',
 'Washington': 'WAS',
 'Pittsburgh': 'PIT',
 'Cincinnati': 'CIN'}

df['team_abb'] = df['Team'].map(mapping)

In [12]:
top_dict = pd.Series(df['2025'].values, index=df['team_abb']).to_dict()

## Add stats for previous weeks in schedule

In [13]:
desired_columns = ['week', 'team', 'opponent_team', 'avg_passing_epa', 'avg_passing_cpoe', 'avg_rushing_epa', 'avg_receiving_epa', 'avg_fg_pct', 'avg_pat_pct', 'avg_completions',
       'avg_attempts', 'avg_passing_yards', 'avg_passing_tds', 'avg_passing_interceptions', 'avg_sacks_suffered', 'avg_sack_yards_lost', 'avg_sack_fumbles_lost', 'avg_carries',
       'avg_rushing_yards', 'avg_rushing_tds', 'avg_rushing_fumbles_lost', 'avg_rushing_first_downs', 'avg_receiving_fumbles_lost', 'avg_def_tackles_solo', 'avg_def_tackles_with_assist',
       'avg_def_tackle_assists', 'avg_def_tackles_for_loss', 'avg_def_tackles_for_loss_yards', 'avg_def_fumbles_forced', 'avg_def_sacks', 'avg_def_sack_yards', 'avg_def_qb_hits',
       'avg_def_interceptions', 'avg_def_interception_yards', 'avg_def_pass_defended', 'avg_def_tds', 'avg_def_fumbles', 'avg_def_safeties', 'avg_penalties', 'avg_penalty_yards',
       'avg_punt_returns', 'avg_punt_return_yards', 'avg_kickoff_returns', 'avg_kickoff_return_yards', 'avg_fg_att']

team_stats = team_stats[desired_columns]

# add 1 week because thats the data we can actually use for the week we predict
team_stats['week'] = team_stats['week']

In [14]:
offensive = ['week', 'team', 'avg_passing_epa', 'avg_passing_cpoe', 'avg_rushing_epa', 'avg_receiving_epa', 'avg_fg_pct', 'avg_pat_pct', 'avg_completions',
       'avg_attempts', 'avg_passing_yards', 'avg_passing_tds', 'avg_passing_interceptions', 'avg_sacks_suffered', 'avg_sack_yards_lost', 'avg_sack_fumbles_lost', 'avg_carries',
       'avg_rushing_yards', 'avg_rushing_tds', 'avg_rushing_fumbles_lost', 'avg_rushing_first_downs', 'avg_receiving_fumbles_lost', 'avg_penalties', 'avg_penalty_yards',
       'avg_punt_returns', 'avg_punt_return_yards', 'avg_kickoff_returns', 'avg_kickoff_return_yards', 'avg_fg_att']

defensive = ['week', 'opponent_team', 'avg_def_tackles_solo', 'avg_def_tackles_with_assist', 'avg_def_tackle_assists', 'avg_def_tackles_for_loss', 'avg_def_tackles_for_loss_yards', 
        'avg_def_fumbles_forced', 'avg_def_sacks', 'avg_def_sack_yards', 'avg_def_qb_hits', 'avg_def_interceptions', 'avg_def_interception_yards', 'avg_def_pass_defended', 
        'avg_def_tds', 'avg_def_fumbles', 'avg_def_safeties']

offensive_stats = team_stats[offensive]
defensive_stats = team_stats[defensive]

In [15]:
# columns from schedule: week, home_team, away_team, home_score, away_score, total_line
schedule = schedule[schedule['week'] > 1]
schedule_interests = schedule[['game_id', 'week', 'home_team', 'home_score', 'away_team', 'away_score', 'total_line']]

In [16]:
schedule_away = schedule_interests
schedule_home = schedule_interests

In [17]:
schedule_away = schedule_away.rename(columns={'home_team':'opponent', 'away_team':'team', 'away_score':'points'})
schedule_away = schedule_away[['game_id', 'week', 'team', 'opponent', 'points', 'total_line']]

schedule_home = schedule_home.rename(columns={'away_team':'opponent', 'home_team':'team', 'home_score':'points'})
schedule_home = schedule_home[['game_id', 'week', 'team', 'opponent', 'points', 'total_line']]

In [18]:
schedule_total = pd.concat([schedule_away, schedule_home])

In [19]:
schedule_total

,game_id,week,team,opponent,points,total_line
16,2025_02_WAS_GB,2,WAS,GB,18.0,48.5
17,2025_02_CLE_BAL,2,CLE,BAL,17.0,46.5
18,2025_02_JAX_CIN,2,JAX,CIN,27.0,49.5
19,2025_02_NYG_DAL,2,NYG,DAL,37.0,44.5
20,2025_02_CHI_DET,2,CHI,DET,21.0,46.5
...,...,...,...,...,...,...
267,2025_18_DAL_NYG,18,NYG,DAL,NaN,NaN
268,2025_18_WAS_PHI,18,PHI,WAS,NaN,NaN
269,2025_18_BAL_PIT,18,PIT,BAL,NaN,NaN
270,2025_18_SEA_SF,18,SF,SEA,NaN,NaN


In [20]:
offensive_stats

,week,team,avg_passing_epa,avg_passing_cpoe,avg_rushing_epa,avg_receiving_epa,avg_fg_pct,avg_pat_pct,avg_completions,avg_attempts,...,avg_rushing_fumbles_lost,avg_rushing_first_downs,avg_receiving_fumbles_lost,avg_penalties,avg_penalty_yards,avg_punt_returns,avg_punt_return_yards,avg_kickoff_returns,avg_kickoff_return_yards,avg_fg_att
0,1,ARI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,ARI,1.524799,-1.733811,1.158540,8.777653,0.666667,1.0,21.000000,29.000000,...,0.000000,6.000000,0.0,9.000000,54.000000,3.000000,30.000000,3.000000,73.000000,3.000000
2,3,ARI,3.832284,-0.485040,0.494152,8.592124,0.833333,1.0,19.000000,27.000000,...,0.000000,5.500000,0.0,10.500000,75.000000,2.000000,29.500000,3.500000,80.000000,2.500000
3,4,ARI,2.348651,-1.198549,0.300113,6.395634,0.777778,1.0,20.000000,29.666667,...,0.000000,6.000000,0.0,8.666667,60.000000,1.666667,22.333333,3.666667,91.666667,2.666667
4,5,ARI,0.935389,-2.138376,-0.249674,6.292581,0.833333,1.0,21.750000,32.500000,...,0.000000,5.000000,0.0,8.250000,56.500000,1.500000,20.500000,4.000000,100.250000,2.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,6,WAS,3.389712,1.695046,2.802895,8.699414,0.750000,1.0,17.800000,29.200000,...,0.400000,7.600000,0.0,7.400000,54.200000,2.400000,35.400000,3.600000,119.600000,2.400000
316,7,WAS,3.897319,3.270358,0.636833,9.342742,0.700000,1.0,18.000000,28.666667,...,0.666667,7.666667,0.0,7.000000,51.833333,2.166667,31.000000,3.666667,120.833333,2.333333
317,8,WAS,0.489855,1.102632,1.373142,6.596160,0.700000,1.0,17.714286,29.142857,...,0.571429,7.714286,0.0,7.571429,61.285714,2.000000,27.714286,4.285714,133.428571,2.000000
318,9,WAS,-0.496898,1.105433,0.950164,5.384997,0.700000,1.0,18.125000,29.250000,...,0.500000,7.250000,0.0,7.000000,58.000000,1.875000,26.000000,4.125000,125.125000,1.750000


In [21]:
schedule_total['team'] = schedule_total['team'].str.strip().str.upper()
offensive_stats['team'] = offensive_stats['team'].str.strip().str.upper()
schedule_total['opponent'] = schedule_total['opponent'].str.strip().str.upper()
defensive_stats['opponent_team'] = defensive_stats['opponent_team'].str.strip().str.upper()

result = schedule_total.merge(offensive_stats, on=['week', 'team'], how='left')
result = result.merge(defensive_stats, left_on=['week', 'opponent'], right_on=['week','opponent_team'], how='left')

C:\Users\tyler\AppData\Local\Temp\ipykernel_43192\2092392173.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  offensive_stats['team'] = offensive_stats['team'].str.strip().str.upper()
C:\Users\tyler\AppData\Local\Temp\ipykernel_43192\2092392173.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  defensive_stats['opponent_team'] = defensive_stats['opponent_team'].str.strip().str.upper()


In [22]:
result['team_top'] = result['team'].map(top_dict)
result['opp_top'] = result['opponent'].map(top_dict)

### Final Touches and Download Data as CSV

In [23]:
# Columns to fill
cols_to_fill = ['avg_fg_pct', 'avg_pat_pct']

# Fill NaN with 1 only for week 2 due to teams not having attempted a fg prior to the week
result.loc[result['week'] == 2, cols_to_fill] = result.loc[result['week'] == 2, cols_to_fill].fillna(1)


In [24]:
result

,game_id,week,team,opponent,points,total_line,avg_passing_epa,avg_passing_cpoe,avg_rushing_epa,avg_receiving_epa,...,avg_def_sack_yards,avg_def_qb_hits,avg_def_interceptions,avg_def_interception_yards,avg_def_pass_defended,avg_def_tds,avg_def_fumbles,avg_def_safeties,team_top,opp_top
0,2025_02_WAS_GB,2,WAS,GB,18.0,48.5,1.364758,-1.289084,7.834049,8.109852,...,11.0,8.0,0.0,0.0,7.0,0.0,0.0,0.0,28:17,30:20
1,2025_02_CLE_BAL,2,CLE,BAL,17.0,46.5,-1.842128,2.374263,-1.793224,2.456954,...,18.0,7.0,0.0,0.0,5.0,0.0,0.0,0.0,30:22,29:03
2,2025_02_JAX_CIN,2,JAX,CIN,27.0,49.5,1.313113,-4.054062,6.582181,3.198194,...,12.0,5.0,2.0,0.0,9.0,0.0,0.0,0.0,31:26,26:49
3,2025_02_NYG_DAL,2,NYG,DAL,37.0,44.5,-11.393837,-13.457901,-3.555864,-4.414557,...,21.0,5.0,0.0,0.0,3.0,0.0,0.0,0.0,30:11,29:08
4,2025_02_CHI_DET,2,CHI,DET,21.0,46.5,-6.988248,-6.666885,2.531189,-2.970347,...,9.0,5.0,1.0,74.0,5.0,1.0,0.0,0.0,31:08,31:49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,2025_18_DAL_NYG,18,NYG,DAL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30:11,29:08
508,2025_18_WAS_PHI,18,PHI,WAS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30:00,28:17
509,2025_18_BAL_PIT,18,PIT,BAL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27:00,29:03
510,2025_18_SEA_SF,18,SF,SEA,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30:09,28:54


In [25]:
import os

try:
    # Works when running as a .py script
    base_dir = os.path.dirname(__file__)
except NameError:
    # Fallback for Jupyter, Python in Excel, or interactive environments
    base_dir = os.getcwd()

data_path = os.path.join(base_dir, '..', 'Data', 'NFLData.csv')

os.makedirs(os.path.dirname(data_path), exist_ok=True)
result.to_csv(data_path, index=False)